Class Labels:
Class 0: cane
Class 1: cavallo
Class 2: elefante
Class 3: farfalla
Class 4: gallina
Class 5: gatto
Class 6: mucca
Class 7: pecora
Class 8: ragno
Class 9: scoiattolo

In [8]:
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
import random  # Add the import for the random module
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Resizing, Rescaling, RandomFlip, RandomRotation, RandomZoom, RandomContrast
from pathlib import Path
from sklearn.metrics import classification_report, confusion_matrix

def seed_everything(seed=42):
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)  # Ensure random seed is set
    os.environ['PYTHONHASHSEED'] = str(seed)

seed_everything()

#!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

#from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir, pred_and_plot

augment = tf.keras.Sequential([
    layers.Resizing(224, 224),
    layers.Rescaling(1./255),
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1)
])

pretrained_model = tf.keras.applications.efficientnet.EfficientNetB7(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='max'
)

pretrained_model.trainable = False

inputs = pretrained_model.input
x = augment(inputs)

x = Dense(128, activation='relu')(pretrained_model.output)
x = BatchNormalization()(x)
x = Dropout(0.45)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.45)(x)

outputs = Dense(10, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer=Adam(0.00001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

checkpoint_path = '/content/drive/MyDrive/models/animals_classification_model_checkpoint.weights.h5'

model.load_weights(checkpoint_path)

from ipywidgets import widgets, VBox, Label, Output, Image
from IPython.display import display
import io
from PIL import Image as PILImage

def preprocess_uploaded_image(uploaded_img):
    img = PILImage.open(io.BytesIO(uploaded_img['content']))
    img = img.resize((224, 224))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def classify_image(uploaded_img):
    img_array = preprocess_uploaded_image(uploaded_img)
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    predicted_prob = np.max(predictions, axis=1)[0]
    return predicted_class, predicted_prob

btn_upload = widgets.FileUpload()
btn_classify = widgets.Button(description="Classify")
out = Output()

def on_classify_button_clicked(b):
    with out:
        out.clear_output()
        if btn_upload.value:
            uploaded_img = next(iter(btn_upload.value.values()))
            predicted_class, predicted_prob = classify_image(uploaded_img)
            print(f"Predicted Class: {predicted_class}")
            print(f"Predicted Probability: {predicted_prob:.2f}")
            img = PILImage.open(io.BytesIO(uploaded_img['content']))
            display(img)
            print(f"Predicted: {predicted_class} ({predicted_prob:.2f})")
        else:
            print("Please upload an image first.")

btn_classify.on_click(on_classify_button_clicked)

VBox([Label('Select your image:'), btn_upload, btn_classify, out])


In [3]:
!pip install ipywidgets
!pip install voila

  Obtaining dependency information for voila from https://files.pythonhosted.org/packages/8f/14/1828ec5e44085f177548b9d4b777ab84c634491cfee5da1724737e95fd49/voila-0.5.7-py3-none-any.whl.metadata
  Obtaining dependency information for jupyter-server<3,>=2.0.0 from https://files.pythonhosted.org/packages/07/46/6bb926b3bf878bf687b952fb6a4c09d014b4575a25960f2cd1a61793763f/jupyter_server-2.14.0-py3-none-any.whl.metadata
  Obtaining dependency information for websockets>=9.0 from https://files.pythonhosted.org/packages/d1/40/6b169cd1957476374f51f4486a3e85003149e62a14e6b78a958c2222337a/websockets-12.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for jupyter-events>=0.9.0 from https://files.pythonhosted.org/packages/a5/94/059180ea70a9a326e1815176b2370da56376da347a796f8c4f0b830208ef/jupyter_events-0.10.0-py3-none-any.whl.metadata
  Obtaining dependency information for jupyter-server-terminals>=0.4.4 from https://files.pythonhosted.org/packages/07/2d/2b32cdbe8d2a602f697a

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\andre\\anaconda3\\Lib\\site-packages\\~mq\\backend\\cython\\context.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [4]:
!jupyter serverextension enable --sys-prefix voila

Enabling: voila
- Writing config: C:\Users\andre\anaconda3\etc\jupyter
    - Validating...
Error loading server extension voila
      X is voila importable?


In [5]:
!voila "/content/drive/MyDrive/Colab Notebooks/Animal Image Classif.ipynb"

'voila' is not recognized as an internal or external command,
operable program or batch file.
